In [673]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.feature_selection import SequentialFeatureSelector, VarianceThreshold, RFE
from sklearn.svm import SVR, SVC

In [674]:
feat_names = []
for i in range(334):
    feat_names.append('feat' + str(i + 1))

df_train = pd.read_csv('train10000.csv', names=feat_names)
df_train_targets = pd.read_csv('train10000_Label.csv', names=['targets'])

df_test = pd.read_csv('test10000.csv', names=feat_names)
df_test_targets = pd.read_csv('test10000_label.csv', names=['targets'])

df_train=(df_train-df_train.min())/(df_train.max()-df_train.min())
df_test=(df_test-df_test.min())/(df_test.max()-df_test.min())

In [675]:
df_train = df_train.join(df_train_targets['targets'])
df_train.head()

,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,feat10,feat11,feat12,feat13,feat14,feat15,feat16,feat17,feat18,feat19,feat20,feat21,feat22,feat23,feat24,feat25,feat26,feat27,feat28,feat29,feat30,feat31,feat32,feat33,feat34,feat35,feat36,feat37,feat38,feat39,feat40,feat41,feat42,feat43,feat44,feat45,feat46,feat47,feat48,feat49,feat50,feat51,feat52,feat53,feat54,feat55,feat56,feat57,feat58,feat59,feat60,feat61,feat62,feat63,feat64,feat65,feat66,feat67,feat68,feat69,feat70,feat71,feat72,feat73,feat74,feat75,feat76,feat77,feat78,feat79,feat80,feat81,feat82,feat83,feat84,feat85,feat86,feat87,feat88,feat89,feat90,feat91,feat92,feat93,feat94,feat95,feat96,feat97,feat98,feat99,feat100,feat101,feat102,feat103,feat104,feat105,feat106,feat107,feat108,feat109,feat110,feat111,feat112,feat113,feat114,feat115,feat116,feat117,feat118,feat119,feat120,feat121,feat122,feat123,feat124,feat125,feat126,feat127,feat128,feat129,feat130,feat131,feat132,feat133,feat134,feat135,feat136,feat137,feat138,feat139,feat140,feat141,feat142,feat143,feat144,feat145,feat146,feat147,feat148,feat149,feat150,feat151,feat152,feat153,feat154,feat155,feat156,feat157,feat158,feat159,feat160,feat161,feat162,feat163,feat164,feat165,feat166,feat167,feat168,feat169,feat170,feat171,feat172,feat173,feat174,feat175,feat176,feat177,feat178,feat179,feat180,feat181,feat182,feat183,feat184,feat185,feat186,feat187,feat188,feat189,feat190,feat191,feat192,feat193,feat194,feat195,feat196,feat197,feat198,feat199,feat200,feat201,feat202,feat203,feat204,feat205,feat206,feat207,feat208,feat209,feat210,feat211,feat212,feat213,feat214,feat215,feat216,feat217,feat218,feat219,feat220,feat221,feat222,feat223,feat224,feat225,feat226,feat227,feat228,feat229,feat230,feat231,feat232,feat233,feat234,feat235,feat236,feat237,feat238,feat239,feat240,feat241,feat242,feat243,feat244,feat245,feat246,feat247,feat248,feat249,feat250,feat251,feat252,feat253,feat254,feat255,feat256,feat257,feat258,feat259,feat260,feat261,feat262,feat263,feat264,feat265,feat266,feat267,feat268,feat269,feat270,feat271,feat272,feat273,feat274,feat275,feat276,feat277,feat278,feat279,feat280,feat281,feat282,feat283,feat284,feat285,feat286,feat287,feat288,feat289,feat290,feat291,feat292,feat293,feat294,feat295,feat296,feat297,feat298,feat299,feat300,feat301,feat302,feat303,feat304,feat305,feat306,feat307,feat308,feat309,feat310,feat311,feat312,feat313,feat314,feat315,feat316,feat317,feat318,feat319,feat320,feat321,feat322,feat323,feat324,feat325,feat326,feat327,feat328,feat329,feat330,feat331,feat332,feat333,feat334,targets
0,1.000000,0.345679,0.000001,0.345679,0.666667,0.000001,1.000000,0.000003,0.000002,1.000000,0.225,0.000,1.00000,0.360902,1.000000,0.014354,0.586930,0.626387,0.462392,0.136364,0.255319,0.454545,0.238872,0.396825,0.082946,0.285714,0.662791,0.005351,0.000001,0.303571,0.426667,0.0806,0.266667,0.2,0.375000,0.571429,0.371429,0.9845,0.9845,0.47500,0.468387,0.555975,0.588679,0.472081,0.15,0.2420,0.2270,0.2470,0.2765,0.2340,0.150286,0.130909,0.2030,0.1330,0.415094,0.615385,0.407407,0.226415,0.9845,0.116994,0.069714,0.097167,0.3125,0.151515,0.493151,0.081081,0.000000,0.111111,0.453488,0.342857,0.060606,0.155172,0.061224,0.0,0.717172,0.260870,0.325,0.000000,0.444444,0.183673,0.709677,0.074074,0.361702,0.192308,0.000000,0.000000,0.000000,0.060606,0.04,0.796875,0.0,0.000000,0.071429,0.020408,0.081633,0.095238,0.00000,0.0,0.000000,0.1250,0.0,0.000000,0.000000,0.101010,0.000000,0.000000,0.000000,0.000000,0.00000,0.263158,0.212121,0.142857,0.210526,0.2,0.172414,0.156863,0.131579,0.0,0.757576,0.131579,0.777778,0.071429,0.029412,0.111111,0.243243,0.00000,0.092593,0.111111,0.086957,0.250,0.047619,0.303030,0.000000,0.20,0.073171,0.393939,0.166667,0.10,0.969697,0.020833,0.594595,0.285714,0.031915,0.310811,0.137931,0.0625,0.044776,0.392857,0.342105,0.222222,0.282828,0.214286,0.060606,0.040404,0.096774,0.757576,0.0,0.646465,0.166667,0.656566,0.000000,0.101010,0.190476,0.0125,0.000000,0.010417,0.0,0.0,0.153846,0.353535,0.10,0.2

In [676]:
df_test = df_test.join(df_test_targets['targets'])
df_test.head()

,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,feat10,feat11,feat12,feat13,feat14,feat15,feat16,feat17,feat18,feat19,feat20,feat21,feat22,feat23,feat24,feat25,feat26,feat27,feat28,feat29,feat30,feat31,feat32,feat33,feat34,feat35,feat36,feat37,feat38,feat39,feat40,feat41,feat42,feat43,feat44,feat45,feat46,feat47,feat48,feat49,feat50,feat51,feat52,feat53,feat54,feat55,feat56,feat57,feat58,feat59,feat60,feat61,feat62,feat63,feat64,feat65,feat66,feat67,feat68,feat69,feat70,feat71,feat72,feat73,feat74,feat75,feat76,feat77,feat78,feat79,feat80,feat81,feat82,feat83,feat84,feat85,feat86,feat87,feat88,feat89,feat90,feat91,feat92,feat93,feat94,feat95,feat96,feat97,feat98,feat99,feat100,feat101,feat102,feat103,feat104,feat105,feat106,feat107,feat108,feat109,feat110,feat111,feat112,feat113,feat114,feat115,feat116,feat117,feat118,feat119,feat120,feat121,feat122,feat123,feat124,feat125,feat126,feat127,feat128,feat129,feat130,feat131,feat132,feat133,feat134,feat135,feat136,feat137,feat138,feat139,feat140,feat141,feat142,feat143,feat144,feat145,feat146,feat147,feat148,feat149,feat150,feat151,feat152,feat153,feat154,feat155,feat156,feat157,feat158,feat159,feat160,feat161,feat162,feat163,feat164,feat165,feat166,feat167,feat168,feat169,feat170,feat171,feat172,feat173,feat174,feat175,feat176,feat177,feat178,feat179,feat180,feat181,feat182,feat183,feat184,feat185,feat186,feat187,feat188,feat189,feat190,feat191,feat192,feat193,feat194,feat195,feat196,feat197,feat198,feat199,feat200,feat201,feat202,feat203,feat204,feat205,feat206,feat207,feat208,feat209,feat210,feat211,feat212,feat213,feat214,feat215,feat216,feat217,feat218,feat219,feat220,feat221,feat222,feat223,feat224,feat225,feat226,feat227,feat228,feat229,feat230,feat231,feat232,feat233,feat234,feat235,feat236,feat237,feat238,feat239,feat240,feat241,feat242,feat243,feat244,feat245,feat246,feat247,feat248,feat249,feat250,feat251,feat252,feat253,feat254,feat255,feat256,feat257,feat258,feat259,feat260,feat261,feat262,feat263,feat264,feat265,feat266,feat267,feat268,feat269,feat270,feat271,feat272,feat273,feat274,feat275,feat276,feat277,feat278,feat279,feat280,feat281,feat282,feat283,feat284,feat285,feat286,feat287,feat288,feat289,feat290,feat291,feat292,feat293,feat294,feat295,feat296,feat297,feat298,feat299,feat300,feat301,feat302,feat303,feat304,feat305,feat306,feat307,feat308,feat309,feat310,feat311,feat312,feat313,feat314,feat315,feat316,feat317,feat318,feat319,feat320,feat321,feat322,feat323,feat324,feat325,feat326,feat327,feat328,feat329,feat330,feat331,feat332,feat333,feat334,targets
0,0.000001,0.222222,0.000002,0.222222,0.788013,0.000000,1.000000,1.0,0.000006,0.000001,0.415,0.000,1.00000,0.165414,0.0,0.092233,0.554871,0.581998,0.420469,0.434783,0.496461,0.653061,0.506400,0.591837,0.173436,0.25,0.837209,0.006721,0.000000,0.353846,0.200000,0.1507,0.433333,0.4,0.25,0.750000,0.255319,0.9920,0.9925,0.46750,0.468387,0.561006,0.579874,0.468274,0.45,0.4305,0.3085,0.4270,0.4895,0.4275,0.2750,0.1435,0.5440,0.1300,0.590909,0.824176,0.577778,0.328125,0.9985,0.121437,0.059982,0.095780,0.121212,0.101010,0.353535,0.040404,0.000000,0.101010,0.525253,0.485714,0.090909,0.26,0.085714,0.000000,0.767677,0.285714,0.131313,0.010204,0.5000,0.765957,0.151515,0.259259,0.400000,0.048780,0.0,0.0,0.022222,0.242424,0.033333,0.430556,0.000000,0.142857,0.236842,0.102041,0.061224,0.095238,0.0,0.0000,0.142857,0.200000,0.000000,0.013699,0.015625,0.393939,0.016667,0.015385,0.000000,0.010204,0.00000,0.119048,0.191919,0.166667,0.166667,0.00,0.068966,0.018182,0.06250,0.0,0.878788,0.127273,0.909091,0.000000,0.047619,0.040404,0.021739,0.036364,0.074074,0.170213,0.347826,0.333333,0.130435,0.166667,0.0,0.307692,0.020202,0.352941,0.277778,0.25,0.979798,0.070423,0.666667,0.285714,0.107692,0.337838,0.43750,0.242424,0.260870,0.047619,0.030303,0.080808,0.329114,0.59375,0.032258,0.076923,0.0,0.833333,0.00,0.686869,0.096774,0.707071,0.000000,0.030303,0.15,0.0250,0.000000,0.000000,0.0,0.0,0.076923,0.171717,0.019231,0.214286,0.563636

In [677]:
df_train.shape, df_test.shape

((10000, 335), (10000, 335))

In [678]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df_train.isnull().sum()

feat1      0
feat2      0
feat3      0
feat4      0
feat5      0
feat6      0
feat7      0
feat8      0
feat9      0
feat10     0
feat11     0
feat12     0
feat13     0
feat14     0
feat15     0
feat16     0
feat17     0
feat18     0
feat19     0
feat20     0
feat21     0
feat22     0
feat23     0
feat24     0
feat25     0
feat26     0
feat27     0
feat28     0
feat29     0
feat30     0
feat31     0
feat32     0
feat33     0
feat34     0
feat35     0
feat36     0
feat37     0
feat38     0
feat39     0
feat40     0
feat41     0
feat42     0
feat43     0
feat44     0
feat45     0
feat46     0
feat47     0
feat48     0
feat49     0
feat50     0
feat51     0
feat52     0
feat53     0
feat54     0
feat55     0
feat56     0
feat57     0
feat58     0
feat59     0
feat60     0
feat61     0
feat62     0
feat63     0
feat64     0
feat65     0
feat66     0
feat67     0
feat68     0
feat69     0
feat70     0
feat71     0
feat72     0
feat73     0
feat74     0
feat75     0
feat76     0
feat77     0

In [679]:
for i in range(df_train.shape[1] - 1):
    feat_name = 'feat' + str(i + 1)
    print(feat_name + ': ', len(df_train[feat_name].unique()))

feat1:  3
feat2:  41
feat3:  8
feat4:  42
feat5:  10
feat6:  3
feat7:  5
feat8:  10
feat9:  10
feat10:  4
feat11:  171
feat12:  9
feat13:  19
feat14:  134
feat15:  3
feat16:  113
feat17:  303
feat18:  262
feat19:  287
feat20:  23
feat21:  1089
feat22:  33
feat23:  969
feat24:  39
feat25:  2535
feat26:  15
feat27:  67
feat28:  2853
feat29:  11
feat30:  49
feat31:  76
feat32:  2385
feat33:  30
feat34:  11
feat35:  24
feat36:  26
feat37:  22
feat38:  62
feat39:  63
feat40:  360
feat41:  335
feat42:  340
feat43:  311
feat44:  339
feat45:  21
feat46:  957
feat47:  798
feat48:  947
feat49:  998
feat50:  948
feat51:  678
feat52:  513
feat53:  841
feat54:  551
feat55:  34
feat56:  71
feat57:  35
feat58:  48
feat59:  63
feat60:  4711
feat61:  5028
feat62:  5544
feat63:  74
feat64:  100
feat65:  53
feat66:  59
feat67:  21
feat68:  100
feat69:  82
feat70:  68
feat71:  99
feat72:  52
feat73:  36
feat74:  40
feat75:  71
feat76:  35
feat77:  28
feat78:  99
feat79:  18
feat80:  49
feat81:  56
feat82:

In [680]:
y_train = df_train['targets'].values
df_train = df_train.drop(columns=['targets'])

y_test = df_test['targets'].values
df_test = df_test.drop(columns=['targets'])

In [681]:
# scaler = StandardScaler()
# df_train_array = scaler.fit_transform(df_train)

selector = VarianceThreshold(threshold=0.01)
print(df_train.shape)
#df_train = pd.DataFrame(df_train_array)
selector.fit_transform(df_train)
feat_mask = selector.get_support()
print(feat_mask)
#print(df_train.shape)

#constant_columns = [column for column in df_train.columns if column not in df_train.columns[selector.get_support()]]

#print(len(constant_columns),df_train.shape)

count = 0
for i in range(len(feat_mask)):
    #print(feat_mask[i])
    if feat_mask[i] == False:
        df_train = df_train.drop(df_train.columns[i - count], axis=1)
        df_test = df_test.drop(df_test.columns[i - count], axis=1)
        count = count + 1

#df_train = df_train.drop(labels=constant_columns, axis=1, inplace=True)
print(df_train.shape, df_test.shape)

# estimator = SVR(kernel="linear")
# selector = RFE(estimator, n_features_to_select=333, step=1)
# selector = selector.fit(df_train, y_train)
# print(selector.support_)

(10000, 334)
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True False False False False  True  True False  True False
 False False  True  True  True  True  True  True  True  True  True False
 False False False False False False False False  True  True False  True
  True  True False False False False False  True False  True False  True
  True  True  True  True False False False  True  True  True  True False
 False False False False False  True  True  True  True  True  True  True
 False False False  True False  True False False False  True False False
 False False  True False False  True False  True False  True  True  True
  True  True  True False False False False  True False False  True False
  True  True  True False False False False False False False  True False
 False False False False  True False  True False  True  True False False
  True  True  True  True  True False False False False False False False
 False False  True False  True  True F

In [682]:
df_train.head()

,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,feat10,feat11,feat12,feat13,feat14,feat15,feat20,feat21,feat23,feat27,feat28,feat29,feat30,feat31,feat32,feat33,feat34,feat35,feat45,feat46,feat48,feat49,feat50,feat56,feat58,feat60,feat61,feat62,feat63,feat64,feat68,feat69,feat70,feat71,feat78,feat79,feat80,feat81,feat82,feat83,feat84,feat88,feat90,feat94,feat99,feat102,feat104,feat106,feat107,feat108,feat109,feat110,feat111,feat116,feat119,feat121,feat122,feat123,feat131,feat137,feat139,feat141,feat142,feat145,feat146,feat147,feat148,feat149,feat159,feat161,feat162,feat164,feat165,feat170,feat172,feat173,feat174,feat175,feat177,feat178,feat179,feat182,feat183,feat184,feat185,feat189,feat190,feat192,feat194,feat197,feat198,feat199,feat200,feat203,feat205,feat206,feat207,feat208,feat209,feat210,feat211,feat212,feat215,feat219,feat222,feat223,feat226,feat227,feat228,feat229,feat231,feat234,feat239,feat240,feat241,feat268,feat269,feat270,feat271,feat272,feat273,feat275,feat276,feat277,feat278,feat280,feat281,feat282,feat286,feat289,feat290,feat291,feat294,feat295,feat296,feat297,feat299,feat302,feat304,feat305,feat306,feat307,feat308,feat309,feat310,feat311,feat312,feat314,feat317,feat323,feat324,feat325,feat327,feat328,feat331,feat332
0,1.000000,0.345679,0.000001,0.345679,0.666667,0.000001,1.000000,0.000003,0.000002,1.000000,0.225,0.000,1.00000,0.360902,1.000000,0.136364,0.255319,0.238872,0.662791,0.005351,0.000001,0.303571,0.426667,0.0806,0.266667,0.2,0.375000,0.15,0.2420,0.2470,0.2765,0.2340,0.615385,0.226415,0.116994,0.069714,0.097167,0.3125,0.151515,0.111111,0.453488,0.342857,0.060606,0.000000,0.444444,0.183673,0.709677,0.074074,0.361702,0.192308,0.060606,0.796875,0.020408,0.000000,0.000000,0.101010,0.000000,0.000000,0.000000,0.00000,0.263158,0.212121,0.156863,0.757576,0.777778,0.071429,0.029412,0.047619,0.166667,0.969697,0.594595,0.285714,0.137931,0.0625,0.044776,0.392857,0.342105,0.166667,0.000000,0.101010,0.0125,0.000000,0.353535,0.267857,0.500000,0.441860,0.37500,1.000000,1.000000,0.949495,0.714286,0.107143,0.066667,0.032787,0.589744,0.333333,0.494949,0.390805,0.959596,0.676768,0.333333,0.010204,0.000000,0.292929,0.090909,0.030303,0.010101,0.868687,0.515152,0.020408,0.146067,0.357143,0.1250,0.717172,0.285714,0.309524,0.719512,0.617021,0.346154,0.000000,0.367347,0.101010,0.181818,0.326531,0.000000,0.515152,0.494949,0.343434,0.191919,0.111111,0.795918,0.135593,0.865979,0.135417,0.626263,0.000000,0.383838,0.206897,0.591837,0.145161,0.061224,0.828283,0.616162,0.141414,0.222222,0.727273,0.300000,0.186441,0.21875,0.767677,0.525253,0.010638,0.391304,0.373737,0.066667,0.636364,0.000000,0.000000,0.070707,1.000000,0.7375,1.000000,0.428571,0.120879,0.355263
1,0.000001,0.395062,0.000000,0.395062,0.666667,0.000000,0.000003,0.000006,0.000004,0.000000,0.175,0.375,0.00003,0.345865,0.000001,0.136364,0.210453,0.190406,0.627907,0.001415,0.000007,0.500000,0.626667,0.0623,0.233333,0.3,0.666667,0.15,0.1935,0.1870,0.2380,0.1730,0.604396,0.415094,0.119024,0.078918,0.097851,0.4125,0.232323,0.202020,0.372093,0.228571,0.181818,0.102041,0.500000,0.163265,0.741935,0.055556,0.382979,0.230769,0.121212,0.218750,0.306122,0.142857,0.013158,0.252525,0.074627,0.076923,0.102041,0.09375,0.263158,0.404040,0.058824,0.727273,0.808081,0.035714,0.058824,0.190476,0.277778,0.979798,0.405405,0.142857,0.103448,0.0625,0.014925,0.607143,0.447368,0.233333,0.166667,0.131313,0.0250,0.111111,0.464646,0.196429,0.409091,0.395349,0.37500,0.777778,0.000000,0.898990,0.642857,0.071429,0.066667,0.016393,0.410256,0.222222,0.333333,0.264368,0.888889,0.434343,0.158730,0.000000,0.010309,0.090909,0.020202,0.000000,0.000000,0.737374,0.242424,0.132653,0.112360,0.357143,0.2500,0.636364,0.285714,0.309524,0.597561,0.478723,0.179487,0.000000,0.306122,0.111111,0.020202,0.867347,0.000000,0.858586,0.151515,0.040404,0.020202,0.010101,0.724490,0.169492,0.814433,0.187500,0.000000,1.000000,0.000000,0.224138,0.040816,0.064516,0.163265,0.808081,0.727273,0.090909,0.292929,0.795455,0.466667,0.389

In [683]:
# for i in df_train.columns:
#     df_train.boxplot(column=[i])
#     plt.show()

In [684]:
# exclude_list = [32, 64, 68, 71, 78, 108, 121, 139, 200, 231, 240, 275, 280, 282, 289, 297, 306, 308, 324, 327]
# for i in exclude_list:
#     featName = 'feat' + str(i)
#     df_train = df_train.drop(columns=[featName])
#     df_test = df_test.drop(columns=[featName])
# df_train.shape

In [685]:
# scaler = StandardScaler()
# X_train = scaler.fit_transform(df_train)
# X_test = scaler.transform(df_test)
# X_train

# y_train = df_train['targets'].values
# X_train = df_train.drop(columns=['targets'])

# y_test = df_test['targets'].values
# X_test = df_test.drop(columns=['targets'])

# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

In [686]:
clf = SVC(kernel='rbf', C=115.02, gamma=0.00091)
clf.fit(X_train, y_train)

SVC(C=115.02, gamma=0.00091)

In [687]:
pred_train = clf.predict(X_train)
pred_test = clf.predict(X_test)

In [688]:
for i in pred_test:
    if i == 1:
        print(i)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [689]:
roc_auc_score(y_train, pred_train)

0.5515951716170406

In [690]:
roc_auc_score(y_test, pred_test)

0.5032279148344806